In [1]:
import importlib
import AirportDataRequester
import CodeConverter
importlib.reload(AirportDataRequester)
importlib.reload(CodeConverter)

import pandas as pd
import json
import time
from datetime import datetime, timedelta
import plotly.graph_objects as go
import plotly.express as px

In [2]:
def pretty_print(jsonData):
    print(json.dumps(jsonData, indent=4))

def today_beginning():
    today = datetime.now()
    today_midnight = datetime(today.year, today.month, today.day, 0, 0, 0)
    return int(time.mktime(today_midnight.timetuple()))

def tomorrow_beginning():
    now = datetime.now()
    tomorrow = now + timedelta(days=1)
    tomorrow_midnight = datetime(tomorrow.year, tomorrow.month, tomorrow.day, 0, 0, 0)
    return int(time.mktime(tomorrow_midnight.timetuple()))

In [3]:
search_airport = 'YSSY'

# Get all incoming and outgoing flights today
syd = AirportDataRequester.AirportDataRequester(search_airport)
df = syd.get_flights_dataframe(today_beginning())
df

,origin,destination,aircraft,airline,flightNumber,scheduled
0,YSSY,YIVL,B350,AMX,AM266,2024-11-25 00:30:00
1,YSSY,OMDB,A388,UAE,EK415,2024-11-25 06:00:00
2,YSSY,NFFN,B38M,FJI,FJ914,2024-11-25 06:00:00
3,YSSY,YMML,A320,JST,JQ501,2024-11-25 06:00:00
4,YSSY,YMAV,A320,JST,JQ603,2024-11-25 06:00:00
...,...,...,...,...,...,...
942,YPAD,YSSY,14Z,QFA,QF7484,2024-11-25 23:15:00
943,YBCG,YSSY,B463,JTE,NC7434,2024-11-25 23:38:00
944,YMML,YSSY,B463,JTE,NC7487,2024-11-25 23:44:00
945,YBBN,YSSY,B462,QFA,QF7419,2024-11-25 23:45:00


In [4]:
# Add column for the aircraft family
converter = CodeConverter.CodeConverter()
df["aircraftFamily"] = df["aircraft"].apply(converter.get_aircraft_family)

# Add column for mode
df['mode'] = df['origin'].apply(lambda x: "takeoff" if search_airport in x else "landing")

# Add column for time only
df['timeScheduled'] = df['scheduled'].apply(lambda x: x.strftime('%I:%M %p'))

# Fill None values with empty string
df.fillna("n/a", inplace=True)

df.sample(5)

,origin,destination,aircraft,airline,flightNumber,scheduled,aircraftFamily,mode,timeScheduled
741,YBBN,YSSY,B738,QFA,QF529,2024-11-25 14:50:00,B737,landing,02:50 PM
103,YSSY,NZQN,A20N,ANZ,NZ232,2024-11-25 09:05:00,A320,takeoff,09:05 AM
770,YMML,YSSY,B738,QFA,QF6111,2024-11-25 16:15:00,B737,landing,04:15 PM
20,YSSY,WADD,A21N,JST,JQ39,2024-11-25 06:35:00,A320,takeoff,06:35 AM
866,PHNL,YSSY,B748,UPS,5X34,2024-11-25 19:22:00,B747,landing,07:22 PM


In [5]:
# Plot aircraft families apperances
top_aircraft = df[df['aircraftFamily'] != 'Unknown'].value_counts('aircraftFamily')[:20]

fig = go.Figure(data=[
    go.Bar(x=top_aircraft.index, y=top_aircraft.values, text=top_aircraft.values, textposition='auto')
])

count = len(top_aircraft) if len(top_aircraft) < 20 else 20

fig.update_layout(
    title=f"Top {count} Aircraft Families Appearing Today at {search_airport}",
    xaxis_title="Aircraft Family",
    yaxis_title="Count",
    template="plotly_white",
    width=1000,
    height=500
)
fig.show()

In [6]:
def plot_activity_by_aircraft_family(df):
    """
    df must be a pandas dataframe containing columns
    * 'scheduled'
    * 'aircraftFamily'
    * 'mode'
    * 'flightNumber'
    * 'timeScheduled'
    * 'origin'
    * 'destination'
    * 'aircraft'
    """

    symbol_map = {
        "takeoff": "triangle-up",
        "landing": "triangle-down"
    }

    fig = px.scatter(
        df,
        x="scheduled",
        y="aircraftFamily",
        color="aircraftFamily",
        symbol="mode",
        symbol_map=symbol_map,
        hover_name="flightNumber",
        title="Flight Schedule by Aircraft Family",
        labels={"scheduled": "Scheduled Time", "aircraftFamily": "Aircraft Family"},
        category_orders={"aircraftFamily": df["aircraftFamily"].value_counts().index},
        custom_data=["flightNumber", "timeScheduled", "origin", "destination", "aircraft"]
    )

    fig.update_traces(
        hovertemplate =
                    "Number: <b>%{customdata[0]}</b><br>" +
                    "Time: <b>%{customdata[1]}</b><br><br>" +
                    "Origin: %{customdata[2]}<br>" +
                    "Destination: %{customdata[3]}" +
                    "<extra></extra>",
        mode='markers',
        marker={'sizemode':'area', 'sizeref':10},
        marker_size=10
    )


    fig.update_layout(
        height=600,
        xaxis_title="Scheduled Time",
        yaxis_title="Aircraft Family",
        template="plotly_white",
        showlegend=False,
        hoverlabel_align = 'right',
    )

    fig.show()

In [7]:
plot_activity_by_aircraft_family(df)

In [8]:
filtered = df[df['aircraftFamily'].isin(['A330', 'B787', 'A350', 'B777', 'A380', 'B747'])]
plot_activity_by_aircraft_family(filtered)